# Project 1: What is labelled data worth to Naive Bayes?
---
###### Student Name(s): Maleakhi Agung Wijaya, Chirag Rao Sahib

## Initialisation

In [1]:
# Library
import pandas as pd
import numpy as np
import random

In [2]:
# Data Path Constant
BREAST_CANCER = "2018S1-proj1_data/breast-cancer-dos.csv"
CAR = "2018S1-proj1_data/car-dos.csv"
HYPOTHYROID = "2018S1-proj1_data/hypothyroid-dos.csv"
MUSHROOM = "2018S1-proj1_data/mushroom-dos.csv"

# Column name for each data set
BREAST_CANCER_COLUMN = ["age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat", "class"]
CAR_COLUMN = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
HYPOTHYROID_COLUMN = ["sex", "on_thyroxine", "query_on_thyroxine", "on_antithyroid_medication", "thyroid_surgery", "query_hypothyroid", "query_hyperthyroid", "pregnant", "sick", "tumor", "lithium", "goitre", "TSH_measured", "T3_measured", "TT4_measured", "T4U_measured", "FTI_measured", "TBG_measured", "class"]
MUSHROOM_COLUMN = ["cap-shape", "cap-surface", "cap-color", "bruises", "odor", "gill-attachment", "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring", "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color", "ring-number", "ring-type", "spore-print-color", "population", "habitat", "class"]

# Other Constant
PRIOR_INDEX = 0
POSTERIOR_INDEX = 1
EPSILON = 0.000001 # Epsilon smoothing

In [22]:
# Used to check algorithm correctness
training_df = pd.DataFrame(data={"Headache": ["severe", "no", "mild", "mild", "severe"], "Sore": ["mild", "severe", "mild", "no", "severe"], "Temperature":["high", "normal", "normal", "normal", "normal"], "Cough": ["yes", "yes", "yes", "no", "yes"], "class":["flu", "cold", "flu", "cold", "flu"]})
training_df

,Cough,Headache,Sore,Temperature,class
0,yes,severe,mild,high,flu
1,yes,no,severe,normal,cold
2,yes,mild,mild,normal,flu
3,no,mild,no,normal,cold
4,yes,severe,severe,normal,flu


In [4]:
test_df = pd.DataFrame(data={"Headache": ["mild", "severe"], "Sore": ["severe", "mild"], "Temperature": ["normal", "high"], "Cough":["no", "no"]})
test_df

,Cough,Headache,Sore,Temperature
0,no,mild,severe,normal
1,no,severe,mild,high


## Preprocess

In [40]:
# This function should open a data file in csv, and transform it into a usable format 
# @param data = csv data that will be opened
# @param columns = new column name for header
# @param eliminate = eliminate the missing/ ? instances (recommended if there are only few ? instances)
# @return df = clean pandas dataframe object
def preprocess(data, columns, eliminate=True):
    # Read and add a header to the data frame
    df = pd.read_csv(data, header=None)
    df.columns = columns
    
    # If the parameter ignore is set to be false then we don't ignore
    if (eliminate):
        # Iterate through the dataframe and only append without missing value
        # Capture the index of one with the missing values
        for index, row in df.iterrows():
            for att in row:
                # If encounter missing values in the data, don't use that
                if (att == "?"):
                    df.drop(index, inplace=True)
                    break
    
    # Return the clean data
    return df

## Train Supervised

In [6]:
# This function should build a supervised NB model and return a count
# @param train_data = training data that are used to create the supervised NB classifier
# @param class_label = column name of the class that we want to classify
# @return count_prior = dictionary describing prior count of the class in training data
# @return count_posterior = dictionary of dictionaries posterior count
def train_count_supervised(train_data, class_label):
    # Calculate prior (dictionary_prior)
    # Initiate python dictionary with the number of class in the training data as it's key
    count_prior = {}
    for unique_class in train_data[class_label].unique():
        count_prior[unique_class] = 0
    
    # Loop through the training data and get how many for every classes instance.
    # Now we have the count prior class that are used for prediction
    for index, row in train_data.iterrows():
        count_prior[row[class_label]] += 1
    
    # Calculate count posterior (dictionary_posterior), the data structure used are dictionary
    # of dictionary of dictionaries
    count_posterior = {}
    
    # Setup the dictionary component
    column_name = list(train_data.columns)
    column_name.remove(class_label)
    for col in column_name:
        count_posterior[col] = {}
        for unique_class in train_data[class_label].unique():
            count_posterior[col][unique_class] = {}
            for unique_col in train_data[col].unique():
                count_posterior[col][unique_class][unique_col] = 0
    
    # Now use the training data to perform calculation
    for index, row in train_data.iterrows():
        for col in column_name:
            count_posterior[col][row[class_label]][row[col]] += 1
            
    return((count_prior, count_posterior))

In [7]:
# This function should build supervised NB model and return a probability
# @param train_data = training data that are used to create the supervised NB classifier
# @param class_label = column name of the class that we want to classify
# @return probability_prior = dictionary describing prior probability of the class in training data,
# @return probability_posterior = dictionary of dictionaries posterior probability
def train_probability_supervised(train_data, class_label):
    (count_prior, count_posterior) = train_count_supervised(train_data, class_label)
    
    # Now calculate the probability of each instances, (i.e. 'Cough': {'flu': {'yes': 3, 'no': 0}, 'cold': {'yes': 1, 'no': 1}}
    # will have P(cough = yes | flu) = 3/3, P(cough = no | flu) = 0/3 and P(cough = yes | cold) = 1/2, P(cough = no | cold) = 1/2
    
    # First calculate the prior probability of the class P(c)
    probability_prior = {}
    sum_instance = sum(count_prior.values())
    for unique_class in train_data[class_label].unique():
        probability_prior[unique_class] = count_prior[unique_class] / sum_instance
        
        # Perform epsilon smoothing
        if (count_prior[unique_class] == 0):
            probability_prior[unique_class] = EPSILON
    
    # Calculate the posterior probability
    probability_posterior = count_posterior
    column_name = list(train_data.columns)
    column_name.remove(class_label)
                
    # Now calculate the posterior probability
    for col in column_name:
        for unique_class in train_data[class_label].unique():
            sum_instance = sum(probability_posterior[col][unique_class].values())
            for unique_col in train_data[col].unique():
                probability_posterior[col][unique_class][unique_col] /= sum_instance
                
                # Perform epsilon smoothing
                if (probability_posterior[col][unique_class][unique_col] == 0):
                    probability_posterior[col][unique_class][unique_col] = EPSILON
                
            
    return((probability_prior, probability_posterior))

## Predict Supervised

In [8]:
# This function should predict the class for a set of instances, based on a trained model 
# @param test_data = data to be tested
# @param train_data = data used for setup such as finding all possible classes
# @param class_label = attribute that we want to classify using naive bayes
# @param model = tuple consisting probability_prior and probability_posterior. 
# Mainly use the train_probability_supervised instead of train_count_supervised
# @return test_class = the class predicted by the naive bayes classifier
def predict_supervised(test_data, train_data, class_label, model):
    prior_probability = model[PRIOR_INDEX]
    posterior_probability = model[POSTERIOR_INDEX]
    test_class = [] # used to capture test result
    
    # Used for calculation purposes
    column_name = list(train_data.columns)
    column_name.remove(class_label)
    
    # Get the answer for every test instance
    for index, row in test_data.iterrows():
        # Initiate dictionary capturing the values calculated by naive bayes model
        test_value = {}
        for unique_class in train_data[class_label].unique():
            test_value[unique_class] = 0

        # Calculate for each class using the naive bayes model (log model for multiplication)
        for unique_class in train_data[class_label].unique():
            test_value[unique_class] = np.log(prior_probability[unique_class])
            for col in column_name:
                test_value[unique_class] += np.log(posterior_probability[col][unique_class][row[col]])
            
        # After calculating all of the possible class, we want to choose the maximum
        maximum_class = (train_data[class_label].unique())[0]
        maximum_value = test_value[maximum_class]
        for key, value in test_value.items():
            if (value > maximum_value):
                maximum_value = value
                maximum_class = key
    
        # Append result
        test_class.append(maximum_class)
    
    # Return the classifier for the class
    return test_class

# Evaluate Supervised

In [9]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate_supervised(true_test_result, predicted_test_result):
    if (len(true_test_result) != len(predicted_test_result)):
        print("Error, different length.")
    else:
        # Measure accuracy
        correct = 0
        for i in range(len(true_test_result)):
            if (true_test_result[i] == predicted_test_result[i]):
                correct += 1
            
        accuracy = correct / len(true_test_result)
        
    return accuracy

## Main Program

In [10]:
# Using the breast cancer data
df_breast_cancer = preprocess(BREAST_CANCER, BREAST_CANCER_COLUMN, eliminate=True)
model_main = train_probability_supervised(df_breast_cancer, "class")
predicted_test_result = predict_supervised(df_breast_cancer, df_breast_cancer, "class", model_main)
print("The accuracy for breast cancer dataset is {}.".format(evaluate_supervised(list(df_breast_cancer["class"]), predicted_test_result)))

# Using car data
df_car = preprocess(CAR, CAR_COLUMN, eliminate=True)
model_main = train_probability_supervised(df_car, "class")
predicted_test_result = predict_supervised(df_car, df_car, "class", model_main)
print("The accuracy for car dataset is {}.".format(evaluate_supervised(list(df_car["class"]), predicted_test_result)))

# Using the breast cancer data
df_hypo = preprocess(HYPOTHYROID, HYPOTHYROID_COLUMN, eliminate=True)
model_main = train_probability_supervised(df_hypo, "class")
predicted_test_result = predict_supervised(df_hypo, df_hypo, "class", model_main)
print("The accuracy for hypothyroid dataset is {}.".format(evaluate_supervised(list(df_hypo["class"]), predicted_test_result)))

# Using the breast cancer data
df_mushroom = preprocess(MUSHROOM, MUSHROOM_COLUMN, eliminate=True)
model_main = train_probability_supervised(df_mushroom, "class")
predicted_test_result = predict_supervised(df_mushroom, df_mushroom, "class", model_main)
print("The accuracy for mushroom dataset is {}.".format(evaluate_supervised(list(df_mushroom["class"]), predicted_test_result)))

The accuracy for breast cancer dataset is 0.7689530685920578.
The accuracy for car dataset is 0.8738425925925926.
The accuracy for hypothyroid dataset is 0.9517799352750809.
The accuracy for mushroom dataset is 0.997165131112686.


## Train Unsupervised

In [54]:
# This function initialise the distribution randomly to the dataframe to begin the unsupervised
# calculation.
# @param dataset = dataframe of the dataset
# @param class_label = class that we will use the classification on
# @return unsupervised_dataset = dataset that have been appended by the distribution columns
def initialise_unsupervised_naive_bayes(dataset, class_label):
    # First remove the class label and put that on the columns so that we can assign a distribution
    class_column = list(dataset[class_label].unique())
    last_class = class_column[-1]
    unsupervised_dataset = dataset.drop(["class"], axis=1)
    
    # Add a column to the dataset according to random distribution (initialisation phase)
    row_instance = unsupervised_dataset.shape[0]
    for unique_class in class_column:
        unsupervised_dataset[unique_class] = [0 for i in range(row_instance)]
    
    # Add random value to the dataset
    for index, row in unsupervised_dataset.iterrows():
        max_probability = 1
        for unique_class in class_column:
            # Assign the remaining probability to the last class
            if (unique_class == last_class):
                unsupervised_dataset.loc[index, unique_class] = max_probability
            else:
                unsupervised_dataset.loc[index, unique_class] = random.uniform(0, max_probability)
                max_probability -= unsupervised_dataset.loc[index, unique_class]
    
    return unsupervised_dataset
    
unsupervised_df = initialise_unsupervised_naive_bayes(training_df, "class")
print(unsupervised_df)

  Cough Headache    Sore Temperature       flu      cold
0   yes   severe    mild        high  0.133589  0.866411
1   yes       no  severe      normal  0.311727  0.688273
2   yes     mild    mild      normal  0.707640  0.292360
3    no     mild      no      normal  0.345900  0.654100
4   yes   severe  severe      normal  0.104684  0.895316


In [59]:
# This function should build an unsupervised NB model and return a count
# @param class_column = possible class name (weak unsupervised model)
# @param attribute_column = attributes that are used for calculation
# @param train_data = training data that are used to create the unsupervised NB classifier (format after running initialise_unsupervised_naive_bayes function)
# @param class_label = column name of the class that we want to classify
# @return count_prior = dictionary describing prior count of the class in training data
# @return count_posterior = dictionary of dictionaries posterior count
def train_count_unsupervised(class_column, attribute_column, train_data , class_label):
    # Calculate prior (dictionary_prior)
    # Initiate python dictionary with the number of class in the training data as it's key
    count_prior = {}
    for unique_class in class_column:
        count_prior[unique_class] = 0
    
    # Loop through the training data and sum the probability
    for index, row in train_data.iterrows():
        for unique_class in class_column:
            count_prior[unique_class] += row[unique_class]
    
    # Calculate count posterior (dictionary_posterior), the data structure used are dictionary
    # of dictionary of dictionaries
    count_posterior = {}
    
    # Setup the dictionary component
    for col in attribute_column:
        count_posterior[col] = {}
        for unique_class in class_column:
            count_posterior[col][unique_class] = {}
            for unique_col in train_data[col].unique():
                count_posterior[col][unique_class][unique_col] = 0
    
    # Now use the training data to perform calculation
    for index, row in train_data.iterrows():
        for col in attribute_column:
            for unique_class in class_column:
                count_posterior[col][unique_class][row[col]] += row[unique_class]
   
    return((count_prior, count_posterior))

attribute_column = list(training_df.columns)
attribute_column.remove("class")
train_count_unsupervised(training_df["class"].unique(), attribute_column, unsupervised_df, "class")

({'cold': 3.396459833903957, 'flu': 1.6035401660960429},
 {'Cough': {'cold': {'no': 0.6540996005775863, 'yes': 2.742360233326371},
   'flu': {'no': 0.3459003994224137, 'yes': 1.2576397666736292}},
  'Headache': {'cold': {'mild': 0.9464594317940587,
    'no': 0.6882732249102493,
    'severe': 1.7617271771996492},
   'flu': {'mild': 1.0535405682059413,
    'no': 0.31172677508975066,
    'severe': 0.2382728228003509}},
  'Sore': {'cold': {'mild': 1.1587710543506335,
    'no': 0.6540996005775863,
    'severe': 1.5835891789757373},
   'flu': {'mild': 0.8412289456493666,
    'no': 0.3459003994224137,
    'severe': 0.41641082102426263}},
  'Temperature': {'cold': {'high': 0.8664112231341611,
    'normal': 2.530048610769796},
   'flu': {'high': 0.13358877686583892, 'normal': 1.469951389230204}}})

In [ ]:
# This function should predict the class distribution for a set of instances, based on a trained model
def predict_unsupervised():
    return

In [ ]:
# This function should evaluate a set of predictions, in an unsupervised manner
def evaluate_unsupervised():
    return

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.